In [11]:
import utilities_v2 as ut
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as LR
import os

In [12]:
# Read-in the original information from project file, and seperate each sheet. 
portfolio = pd.read_excel('PortfolioHoldings.xlsx', 'Portfolio', keep_default_na=False)[['Ticker', 'Wt']].iloc[:3,:]
benchmark = pd.read_excel('PortfolioHoldings.xlsx', 'Benchmark', keep_default_na=False)

In [13]:
excel_filename = 'ProjData_1.xlsx'
sheet_name = 'Portfolio'

# build full path of the file using my current working directory
full_path = os.path.join(os.getcwd(), excel_filename)

# add/overwrite sheet
ut.excel_add_sheet(portfolio, full_path, sheet_name, index=False)
sheet_name_2 = 'Benchmark'
ut.excel_add_sheet(benchmark, full_path, sheet_name_2, index=False)

In [14]:
# read it back using pandas
df_port2 = pd.read_excel(full_path, sheet_name, index_col=None, keep_default_na=False)
df_bench2 = pd.read_excel(full_path, sheet_name_2, index_col=None, keep_default_na=False)

In [15]:
# Removing the empty cells
df_port2 = df_port2[['Ticker', 'Wt']].iloc[:3,:]

# Combining the portfolio and benchmark
df_combined = pd.merge(left=df_port2, left_on='Ticker', right=df_bench2, right_on='Ticker', how='outer')

# Rename columns
df_combined = df_combined = df_combined.rename(columns={'Wt_x':'PortWt', 'Wt_y':'BmkWt'})

# Fill out NA cells as 0s
df_combined = df_combined.fillna(0)

# Calculate the active weight
df_combined['ActWt'] = df_combined.apply(lambda row: row.PortWt - row.BmkWt, axis=1)

# Add "MergedHoldings"
ut.excel_add_sheet(df_combined, full_path, 'MergedHoldings', index=False)

In [16]:
# Get ticker names
tickers = df_port2['Ticker'].unique()
b_tickers = df_bench2['Ticker'].unique()

sets = {}

## Get the returns of both the portfolio and the benchmark
## Save them into a dictionary
for t in tickers:
    sets[t] = ut.get_yahoo_returns_monthly(str(t), '2015-10', '2020-09') 

for t in b_tickers:
    sets[t] = ut.get_yahoo_returns_monthly(str(t), '2015-10', '2020-09') 

In [17]:
# Put together the Yahoo returns to a single Dataframe
# 1. use a for-loop to align the formatting
all_returns = []
for item in sets.keys():
    n = sets[item].reset_index()
    n.insert(0, 'Ticker', str(item))
    all_returns.append(n)

# 2. concat everything
returns_df = pd.concat(all_returns)
returns_df

# 3. save it into "SecurityReturns"
ut.excel_add_sheet(returns_df, full_path, 'SecurityReturns', index=False)

In [18]:
# Get FF3FactorReturns
import pandas_datareader as web
source_df = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start='2015-10', end='2020-09')[0].reset_index()

# 3. save it into "FF3FactorReturns"
ut.excel_add_sheet(source_df, full_path, 'FF3FactorReturns', index=False)

In [19]:
# Read the return data stored in previous worksheet
securities_ret = pd.read_excel('ProjData_1.xlsx', 'SecurityReturns')
ff3_ret = pd.read_excel('ProjData_1.xlsx', 'FF3FactorReturns')

# Get what the tickers are for both portfolios and benchmark
# All individual return Dataframe will be saved to main

main = {}
tickers = pd.read_excel('ProjData_1.xlsx', 'MergedHoldings')['Ticker'].values

# Add Merged holdings and returns to main
for t in tickers:
    main[t] = securities_ret.loc[securities_ret['Ticker'] == t]

# Regression of each stock against FFF3 factor
# 1. Setup Linear Regression object, as well as a dictionary to save results
reg = LR()
results = {}

# 2. Setup Independent variable of FF3 factors. 
X = ff3_ret.iloc[:, 1:-1]


#3. Loop over "main" to do the regression for each ticker
#   Get the risk free rate so that dependent variable would be excess
rf = ff3_ret.iloc[:, -1].values
for ticker in main:
    ret = main[ticker]['MthlyRet']
    y = ret - rf
    reg.fit(X, y)
    y_pred = reg.predict(X)
    residual = y - y_pred
    results[ticker] = [reg.coef_ , reg.intercept_, np.std(residual), np.std(y)]


# Make the "betas" and the "residvol" results in to a dataframe
# 1. Make the related metrics of each stock into ONE list, so that Pandas 
#    could process them all in to different columns. 

d = {}
for key in results.keys():
    # A new list that takes all betas and the residual together
    new = []
    
    # Get everything into new
    for num in results[key][0]:
        new.append(num)
    new.append(float(results[key][1]))
    new.append(float(results[key][2]))
    new.append(float(results[key][3]))
    
    # Replace the arrays with one single list. 
    d[key] = new

# 2. Save everything into a dataframe
FF3SecurityData = pd.DataFrame.from_dict(d, orient='index', columns = ['β_Mkt-RF', 'β_SMB', 'β_HML', 'Alpha', 'ResidStdev', 'TotStdev']).reset_index()
FF3SecurityData.rename(columns={'index': 'Ticker'}, inplace=True)

# Write Regression Results into ProjData_1.xlsx
full_path = os.path.join(os.getcwd(), 'ProjData_1.xlsx')
ut.excel_add_sheet(FF3SecurityData, full_path, 'FF3SecurityData', index=False)

# Get each factor's volatility
factors = {}
names = ff3_ret.columns[1:-1]
for k in names:
    factors[k] = np.std(ff3_ret[k], ddof=1)

# Create a dataframe with two columns
FF3FactorVol = pd.DataFrame.from_dict(factors, orient='index', columns = ['ff3_factor_vol']).reset_index()
FF3FactorVol.rename(columns={'index': 'ff3_factor_name'}, inplace=True)

# Add the sheet into excel
ut.excel_add_sheet(FF3FactorVol, full_path, 'FF3FactorVol', index=False)

# Get correlation matrix for the 3 factors
ff3 = ff3_ret.iloc[:, 1:-1]
ff3_Rho = ff3.corr()
ut.excel_add_sheet(ff3_Rho, full_path, 'FF3Correlation', index=False)

In [20]:
# Retreive all information needed for portfolio algebra
holdings = pd.read_excel('ProjData_1.xlsx', 'MergedHoldings')
ff3_reg = pd.read_excel('ProjData_1.xlsx','FF3SecurityData')
ff3_vol =pd.read_excel('ProjData_1.xlsx','FF3FactorVol')
ff3_corr = pd.read_excel('ProjData_1.xlsx','FF3Correlation')

# Get tickers of the portfolio
# This should work even if the ticker name changes 
portfolio = holdings.loc[holdings['PortWt'] != 0]
portfolio_wt = portfolio['PortWt'].values
portfolio_wt

# Get beta by indexing out the portfolio tickers first
port_stat = ff3_reg.loc[ff3_reg['Ticker'].isin(portfolio['Ticker'].values)]
b = port_stat.iloc[:, 1:4]
beta = b.values.T

# Get return standard deviation
resid = port_stat['ResidStdev'].values
sig_resid = np.diag(resid)

# Get factor standard deviation
ff3_stdev = ff3_vol['ff3_factor_vol'].values
sig_F = np.diag(ff3_stdev)

#Get factor correlation
Rho_F = ff3_corr.values
Rho_F

PortVAR_Sys = (portfolio_wt.T @ beta.T) @ (sig_F @ Rho_F @ sig_F) @ (beta @ portfolio_wt)
p_ann_sys_std = (PortVAR_Sys * 12) **0.5

PortVAR_Idiosyncratic = portfolio_wt.T @ (sig_resid @ sig_resid) @ portfolio_wt
p_ann_res_std = (PortVAR_Idiosyncratic *12) ** 0.5
p_ann_tot_std = ((PortVAR_Sys + PortVAR_Idiosyncratic) * 12) ** 0.5
p_sys_per = (PortVAR_Sys)/ (PortVAR_Sys+PortVAR_Idiosyncratic) 
p_res_per = (PortVAR_Idiosyncratic)/ (PortVAR_Sys + PortVAR_Idiosyncratic)

# Get Benchmark Risk, start with the weight
benchmark = holdings.loc[holdings['BmkWt'] != 0]
benchmark_wt = benchmark['BmkWt'].values
benchmark_wt

# Get benchmark beta by indexing out the portfolio tickers first
bmk_stat = ff3_reg.loc[ff3_reg['Ticker'].isin(benchmark['Ticker'].values)]
bmk_b = bmk_stat.iloc[:, 1:4]
bmk_beta = bmk_b.values.T

# Get benchmark return standard deviation
bmk_resid = bmk_stat['ResidStdev'].values
bmk_sig_resid = np.diag(bmk_resid)

# Using the factor standev and rho that has been extracted
BmkVar_Sys = (benchmark_wt.T @ bmk_beta.T) @ (sig_F @ Rho_F @ sig_F) @ (bmk_beta @ benchmark_wt)
b_ann_sys_std = (BmkVar_Sys * 12) **0.5

BmkVar_Idiosyncratic = benchmark_wt.T @ (bmk_sig_resid @ bmk_sig_resid) @ benchmark_wt
b_ann_res_std = (BmkVar_Idiosyncratic * 12) ** 0.5
b_ann_tot_std = ((BmkVar_Idiosyncratic+BmkVar_Sys) * 12) ** 0.5

b_var_tot = BmkVar_Sys+ BmkVar_Idiosyncratic
b_sys_per = BmkVar_Sys/b_var_tot
b_res_per = BmkVar_Idiosyncratic/b_var_tot

# Get active weights
active = holdings.loc[holdings['ActWt'] != np.nan]
active_wt = active['ActWt'].values
active_wt

# Get active beta
act_stat = ff3_reg.loc[ff3_reg['Ticker'].isin(active['Ticker'].values)]
act_b = act_stat.iloc[:, 1:4]
act_beta = act_b.values.T

# Get benchmark return standard deviation
act_resid = act_stat['ResidStdev'].values
act_sig_resid = np.diag(act_resid)

act_sig_resid

# Using the factor standev and rho that has been extracted
ActVar_Sys = (active_wt.T @ act_beta.T) @ (sig_F @ Rho_F @ sig_F) @ (act_beta @ active_wt)
a_ann_sys_std = (ActVar_Sys * 12) **0.5

ActVar_Idiosyncratic = active_wt.T @ (act_sig_resid @ act_sig_resid) @ active_wt
a_ann_res_std = (ActVar_Idiosyncratic * 12) ** 0.5
a_ann_tot_std = ((ActVar_Idiosyncratic+ActVar_Sys) * 12) ** 0.5

a_var_tot = ActVar_Sys+ ActVar_Idiosyncratic
a_sys_per = ActVar_Sys/a_var_tot
a_res_per = ActVar_Idiosyncratic/a_var_tot

summary_dict = {
      #'-- Risk Summary --'             : ['PortWt', 'BmkWt', 'ActWt'],
     'Total Risk (Ann Stddev)'       : [p_ann_tot_std, b_ann_tot_std, a_ann_tot_std]
     ,'Systematic Risk (Ann Stddev)'  : [p_ann_sys_std, b_ann_sys_std, a_ann_sys_std]
     ,'Unsystematic Risk (Ann Stddev)': [p_ann_res_std, b_ann_res_std, a_ann_res_std]
     ,'Sys Risk - % of Total'     : [p_sys_per, b_sys_per, a_sys_per]
     ,'Unsys Risk - % of Total'   : [p_res_per, b_res_per, a_res_per]
  }

df_summary = pd.DataFrame.from_dict(data=summary_dict, orient='index', columns = ['PortWt', 'BmkWt', 'ActWt'])
df_summary = df_summary.round(3)
print('Source data from Yahoo finance and https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html')
print('All calculation based on data of the past 60 months.')
print(df_summary)

full_path = os.path.join(os.getcwd(), 'ProjData_1.xlsx')
ut.excel_add_sheet(df_summary, full_path, 'RiskSummary', index=True, header=True)

Source data from Yahoo finance and https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
All calculation based on data of the past 60 months.
                                PortWt   BmkWt   ActWt
Total Risk (Ann Stddev)         20.884  15.747  10.995
Systematic Risk (Ann Stddev)    17.180  13.465   4.111
Unsystematic Risk (Ann Stddev)  11.873   8.165  10.198
Sys Risk - % of Total            0.677   0.731   0.140
Unsys Risk - % of Total          0.323   0.269   0.860
